# Emulation of nonlinear audio systems with CNN in Real Time
version : LSTM9

Imports :

In [1]:
import sys,select,os
sys.path.append('./Codes')
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
import scipy.io.wavfile
import time
import pyaudio 
from tensorflow.contrib.rnn import *

pathModel = './Experiments/2018-09-24-14-00/temp/' #(200,150) dnn 150 step



if not os.path.isdir(pathModel):
    print("could nt find path model {}".format(pathModel))
num_step = 150
bufferAudioSize = 300
fs = 44100

with tf.Session() as sess:
    saver = tf.train.import_meta_graph(pathModel+'myModel.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint(pathModel))
    lastBuff = np.zeros((num_step-1,))
    graph = tf.get_default_graph()
    data = graph.get_tensor_by_name("placeHolder/data:0")
    prediction = graph.get_tensor_by_name("FCPred/Tanh:0")
    
    dataNonShaped = tf.placeholder(tf.float32, [bufferAudioSize,],name="inputFromADC")
    prevBuff = tf.placeholder(tf.float32, [num_step-1,],name = "EndofPreviousInputBufferNonShaped")
    newBuff = tf.concat([prevBuff,dataNonShaped],0) #size num_step-1+bufferAudioSize
    
    nextBuff = newBuff[-(num_step-1):] # save for next iteration
    
    my_indices = tf.constant(np.arange(bufferAudioSize))
    
    indices = (np.arange(num_step) +my_indices[:,tf.newaxis])
    dataShaped = tf.gather(newBuff,indices) # slice input vector into tensor of shape(audiobufferSize,num_step)

    def callback(in_data, frame_count, time_info, flag):
        if flag:
            print("Playback Error: {}".format(flag))
        global lastBuff

        audio_data = np.frombuffer(in_data, dtype=np.float32)
        dataShapedToProcess,lastBuff = sess.run([dataShaped,nextBuff], feed_dict={dataNonShaped : audio_data, prevBuff : lastBuff})
        out = sess.run(prediction, feed_dict={data: dataShapedToProcess})
        return out, pyaudio.paContinue
    
    #fake graph use before launch audio stream, to place node on the graph
    for x in range(0, 5):
        initInput = [0]*bufferAudioSize
        dataShapedToProcess = sess.run([dataShaped], feed_dict={dataNonShaped : initInput, prevBuff : lastBuff})
        _ = sess.run(prediction, feed_dict={data: dataShapedToProcess[0]})
    
    pa = pyaudio.PyAudio()
    
    stream = pa.open(format = pyaudio.paFloat32,
                     channels = 1,
                     rate = fs,
                     output = True,
                     input = True,
                     frames_per_buffer = bufferAudioSize,
                     stream_callback = callback)
    print("input latency {} s".format(stream.get_input_latency()))
    print("output latency {} s".format(stream.get_output_latency()))

    #stream.start_stream()
    while stream.is_active():   
        time.sleep(0.1)

    print("stream interrupted")   
    stream.stop_stream()
    stream.close()
    pa.terminate()

INFO:tensorflow:Restoring parameters from ./Experiments/2018-09-24-14-00/temp/myFinalModel.ckpt
input latency 0.011609977324263039 s
output latency 0.017414965986394557 s
Playback Error: 4
Playback Error: 4
stream interrupted
